<a href="https://colab.research.google.com/github/cristiandarioortegayubro/BDS/blob/main/modulo.08/bds_aprendizaje_profundo_001_02_Backpropagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20BDS%20Horizontal%208.png?raw=true">
</p>

 # **<font color="DeepPink">Backpropagation</font>**

Ejemplo extraído de https://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/

In [ ]:
from random import seed
from random import random
from math import exp

In [ ]:
# Generamos una red neuronal sencilla con 1 capa o layer
# Function takes as inputs: number of neurons, numbers of hidden layers, number of categories (outputs)
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

In [ ]:
# Generamos una red neuronal sencilla con 1 capa y 1 neurona en dicha capa
# La hidden layer tiene 2 weights y el bias, mientras que la output layer tiene en cada unidad 1 weight y 1 bias
seed(1)
network = initialize_network(2, 1, 2)
for layer in network:
    print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]


En el siguiente paso, desarrollamos una función para activar la red neuronal

In [ ]:
# Se "activa" la neurona de una red a partir de los weights y los inputs
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation

Se utiliza la función sigmoide como función de activación de las neuronas

In [ ]:
# Se utiliza la función sigmoide como función de activación
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))

Comenzamos con el paso de "forward propagation", que es el paso desde la capa de inputs hasta la de outputs

In [ ]:
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

# test forward propagation
network = [[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
           [{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]]
row = [1, 0, None] # Valores iniciales de los weights y bias de la neurona en la hidden layer
output = forward_propagate(network, row)
print(output)

[0.6629970129852887, 0.7253160725279748]


Vamos ahora al paso de Backpropagation, donde el error de predicción es propagado hacia atrás en la red neuronal

In [ ]:
# Por lo tanto, la función para la derivada de la función sigmoide será:

def transfer_derivative(output):
    return output * (1.0 - output)

In [ ]:
# Definimos a su vez la función para "propagar hacia atrás" el error
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                    errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(neuron['output'] - expected[j])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

In [ ]:
# Testeamos el backpropagation
network = [[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
[{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095]}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763]}]]
expected = [0, 1]
backward_propagate_error(network, expected)
for layer in network:
    print(layer)

[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'delta': 0.007668854370284511}]
[{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095], 'delta': 0.14619064683582808}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763], 'delta': -0.0771723774346327}]


In [ ]:
# Hacemos el update de los weights
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] -= l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] -= l_rate * neuron['delta']

In [ ]:
# Finalmente, se entrena el modelo para un número dado de epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [ ]:
# Testeamos la red construida de esta forma para un ejemplo particular.
seed(1)
dataset = [[2.7810836,2.550537003,0],
 [1.465489372,2.362125076,0],
 [3.396561688,4.400293529,0],
 [1.38807019,1.850220317,0],
 [3.06407232,3.005305973,0],
 [7.627531214,2.759262235,1],
 [5.332441248,2.088626775,1],
 [6.922596716,1.77106367,1],
 [8.675418651,-0.242068655,1],
 [7.673756466,3.508563011,1]]
n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
network = initialize_network(n_inputs, 2, n_outputs)
train_network(network, dataset, 0.5, 20, n_outputs)
for layer in network:
    print(layer)

>epoch=0, lrate=0.500, error=6.365
>epoch=1, lrate=0.500, error=5.557
>epoch=2, lrate=0.500, error=5.291
>epoch=3, lrate=0.500, error=5.262
>epoch=4, lrate=0.500, error=5.217
>epoch=5, lrate=0.500, error=4.899
>epoch=6, lrate=0.500, error=4.419
>epoch=7, lrate=0.500, error=3.900
>epoch=8, lrate=0.500, error=3.461
>epoch=9, lrate=0.500, error=3.087
>epoch=10, lrate=0.500, error=2.758
>epoch=11, lrate=0.500, error=2.468
>epoch=12, lrate=0.500, error=2.213
>epoch=13, lrate=0.500, error=1.989
>epoch=14, lrate=0.500, error=1.792
>epoch=15, lrate=0.500, error=1.621
>epoch=16, lrate=0.500, error=1.470
>epoch=17, lrate=0.500, error=1.339
>epoch=18, lrate=0.500, error=1.223
>epoch=19, lrate=0.500, error=1.122
[{'weights': [-0.9766426647918854, 1.0573043092399, 0.7999535671683315], 'output': 0.05429927062285241, 'delta': 0.0035328621774792703}, {'weights': [-1.2245133652927975, 1.4766900503308025, 0.7507113892487565], 'output': 0.03737569585208105, 'delta': 0.005989297622698788}]
[{'weights': [1

<br>
<br>
<p align="center"><b>
💗
<font color="DeepPink">
Hemos llegado al final de nuestro colab...
</font>
</p>
<br>
<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20BDS%20Horizontal%208.png?raw=true">
</p>

---